<a href="https://colab.research.google.com/github/ChickenRob0t/W_Scrapping_Meteorologia/blob/main/PCYB_Puntos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar e importar todas las dependencias
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

import pandas as pd
import time

In [ ]:
#Código para eliminar muchos problemas con Selenium (Esto sólo aplica en google colab)
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

#https://github.com/googlecolab/colabtools/issues/3347 Para recordar xd, sirve en mayo :p

In [ ]:
fecha_actual = datetime.now()                   #Problemas técnicos con la zona horaria para que me entregue el día de hoy siempre
desplazamiento_gdl = timedelta(hours=-6)

date_p= (fecha_actual + desplazamiento_gdl ).strftime('%Y-%m-%d')

In [ ]:
def datos_pluv(n_clicks=False):

    options = webdriver.ChromeOptions()
    options.add_argument("--prefs")
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")                           #Configuración general para entrar a la página

    URL="https://uepcbj.com/uepcb_clp/map_visitors.php"                      #------------------
    driver=webdriver.Chrome(options=options)                                      #--------
    driver.get(URL)                                                                 #---

    zoom=driver.find_element(By.CLASS_NAME, 'leaflet-control-zoom-in') #Botón de zoom en la página
    if n_clicks:              #Si pide que le des click al botón de zoom
        zoom.click()

    Colores=driver.find_elements(By.CLASS_NAME,'leaflet-interactive') #Entrega la información de los puntitos
    base=pd.DataFrame({'Estado':[] , 'Municipio':[] ,'Dependencia':[] , 'Colonia':[] , 'Lectura':[] }) #Generamos la base
                                                                                                    #de datos
    for color in Colores:                       #Configuración para clickear cada botón, entrega la información y la
        try:                                                   #almacena en la base de datos
            color.click()
            info = (driver.find_element(By.CLASS_NAME, 'leaflet-popup-content').text).split('\n')
            fila= dict(map(lambda x: x.split(':'),info))
            base=base.append(fila,ignore_index=True)
            time.sleep(1)
        except:
            pass


    return base

In [ ]:
w_c=datos_pluv(False)                     #Genera uno donde sí hace zoom, luego donde no
w_oc=datos_pluv(True)        #Aquí sí da zoom

completo=pd.concat([w_c,w_oc]).drop_duplicates()        #Los une y elimina los que se repitan
completo

In [ ]:
completo.to_csv('Datos_pluviales_{}.csv'.format(date_p)) #Lo guarda como csv para ya poderlo descargar